In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Paramètres de vol

In [ ]:
Vb = 2
Diametre_bouteille = 8.9
Diametre_sortie = 0.9
Masse_fusee_vide = 0.5
Cx = 0.1
a = 89 # prévoir de le transformer en radians pour utiliser les fonctions trigo
lr = 22
Po = 10
Patm = 101325
Veo = 0.65

In [ ]:
# Conversion des volumes en mètres cubes
Vb /= 1000
Veo /= 1000
# Conversion de la pression en pascal
Po *= 100000

## 1ère colonne : Volume air (m^3)

In [ ]:
volumes_air = [Vb - Veo]
volumes_air_final = (Po + Patm)*(Vb-Veo)/Patm

# Premiere phase
for i in range(28) :
    volumes_air.append(volumes_air[i] + (Vb-volumes_air[0])/29)
# Fin premiere phase 
volumes_air.append(Vb)
volumes_air.append(Vb)
# Début deuxième phase
for i in range(30,48) :
    volumes_air.append(volumes_air[i] + (volumes_air_final-volumes_air[30])/19)
# Fin deuxième phase
volumes_air.append(volumes_air_final)
# Fin vecteur

## 2eme colonne : Pression air en Pascal

In [ ]:
pression_air = list()
for i in range(len(volumes_air)) : 
    pression_air.append(((Po + Patm)*(Vb-Veo)/volumes_air[i])-Patm)